# Justifying a random forest's predictions
## Libraries and experimental data set

In [1]:
# Libraries to be used
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from lime.lime_tabular import LimeTabularExplainer
from scipy.sparse import hstack

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Load data set for experiments
from sklearn.datasets import load_wine
dataset = load_wine()
X = pd.DataFrame(dataset.data, columns = dataset.feature_names)
y = dataset.target

In [3]:
# Split data into training and test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    stratify = y,
    train_size = 100,
    random_state = 2018   # for reproducibility
)

## Exploratory analysis and random forest classifiers

In [4]:
# Take a glance at feature distributions, broken down by class
Xtrain.groupby(ytrain).quantile([.1,.9])

alcohol  malic_acid    ash  alcalinity_of_ash  magnesium  \
0 0.1   13.208       1.640  2.178               14.0       96.2   
  0.9   14.356       2.998  2.700               20.4      118.0   
1 0.1   11.787       0.976  1.746               16.0       80.0   
  0.9   13.133       3.430  2.676               25.1      110.2   
2 0.1   12.486       2.146  2.200               18.5       87.2   
  0.9   13.980       4.986  2.694               25.0      111.4   

       total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
0 0.1          2.600       2.618                 0.202            1.352   
  0.9          3.294       3.546                 0.340            2.380   
1 0.1          1.450       1.250                 0.237            0.820   
  0.9          2.842       2.671                 0.532            1.992   
2 0.1          1.322       0.522                 0.282            0.836   
  0.9          2.240       1.134                 0.618            1.560   

       color_intensity    hue  od280/od315_of_diluted_wines  proline  
0 0.1            4.332  0.882                         2.754    845.0  
  0.9            7.250  1.224                         3.518   1474.0  
1 0.1            1.950  0.898                         2.047    364.0  
  0.9            3.870  1.360                         3.217    710.8  
2 0.1            4.912  0.566                         1.348    501.0  
  0.9           10.364  0.832                         1.976    832.0

In [5]:
# Train a random forest classifier for each of the three classes
clf = []
for clas in range(3):
    clf.append(
        RandomForestClassifier(
            n_estimators = 100, n_jobs = -1,
            random_state = 2018   # for reproducibility
        ).fit(Xtrain, ytrain == clas)
    )

In [6]:
# Check the AUC's of the classifiers on the test data
for clas in range(3):
    print(roc_auc_score(ytest == clas, clf[clas].predict_proba(Xtest)[:,1]))

1.0
0.994509265614
1.0


## Explanations from LIME

In [7]:
# Create a LIME explainer for tabular data
explainer = LimeTabularExplainer(
    Xtrain.values, feature_names = Xtrain.columns,
    random_state = 2018   # for reproducibility
)

def explain_row(clf, row, num_reasons = 2):
    '''
    Produce LIME explanations for a single row of data.
        * `clf` is a binary classifier (with a predict_proba method),
        * `row` is a row of features data,
        * `num_reasons` (default 2) is the number of 
          reasons/explanations to be produced.
          
    '''
    exp = [
        exp_pair[0] for exp_pair in     # Get each explanation (a string)
        explainer.explain_instance(     # from the LIME explainer
            row, clf.predict_proba,     # for the given row and classifier
            labels = [1],               # and label 1 ("positives")
            num_features = num_reasons  # for up to `num_reasons` explanations
        ).as_list()
        if exp_pair[1] > 0              # but only for positive explanations 
    ][:num_reasons]
    
    # Fill in any missing explanations with blanks
    exp += [''] * (num_reasons - len(exp))  
    return exp


def predict_explain(rf, X, num_reasons = 2):
    '''
    Produce scores and LIME explanations for every row in a data frame.
        * `rf` is a binary classifier with a predict_proba method,
        * `X` is the features data frame,
        * `num_reasons` (default 2) is the number of 
          reasons/explanations to be produced for each row.
          
    '''
    # Prepare the structure to be returned
    pred_ex = Xtest[[]]
    
    # Get the scores from the classifier
    pred_ex['SCORE'] = rf.predict_proba(X)[:,1]
    
    # Get the reasons/explanations for each row
    cols = zip(
        *Xtest.apply(
            lambda x: explain_row(rf, x, num_reasons), 
            axis = 1, raw = True
        )
    )
    
    # Return the results
    for n in range(num_reasons):
        pred_ex['REASON%d' % (n+1)] = next(cols)
    return pred_ex


### Explanations for top cases predicted to belong to class 0

In [8]:
%%time
pe0l = predict_explain(clf[0], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 44 s


In [9]:
pe0l

,SCORE,REASON1,REASON2,TRUE_CLASS
58,1.00,proline > 973.75,flavanoids > 2.81,0
57,1.00,proline > 973.75,flavanoids > 2.81,0
0,1.00,proline > 973.75,flavanoids > 2.81,0
26,0.98,proline > 973.75,flavanoids > 2.81,0
49,0.98,proline > 973.75,flavanoids > 2.81,0
31,0.98,proline > 973.75,flavanoids > 2.81,0
51,0.96,proline > 973.75,flavanoids > 2.81,0
47,0.96,proline > 973.75,flavanoids > 2.81,0
9,0.95,proline > 973.75,flavanoids > 2.81,0
13,0.95,proline > 973.75,flavanoids > 2.81,0


### Explanations for top cases predicted to belong to class 1

In [10]:
%%time
pe1l = predict_explain(clf[1], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 45.4 s


In [11]:
pe1l

,SCORE,REASON1,REASON2,TRUE_CLASS
128,1.00,color_intensity <= 3.14,proline <= 495.00,1
97,1.00,color_intensity <= 3.14,proline <= 495.00,1
86,1.00,color_intensity <= 3.14,alcohol <= 12.41,1
117,0.99,color_intensity <= 3.14,proline <= 495.00,1
113,0.99,color_intensity <= 3.14,alcohol <= 12.41,1
106,0.99,alcohol <= 12.41,ash <= 2.20,1
114,0.99,color_intensity <= 3.14,proline <= 495.00,1
78,0.98,alcohol <= 12.41,malic_acid <= 1.64,1
99,0.98,color_intensity <= 3.14,proline <= 495.00,1
94,0.98,alcohol <= 12.41,proline <= 495.00,1


### Explanations for top cases predicted to belong to class 2

In [12]:
%%time
pe2l = predict_explain(clf[2], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 45.1 s


In [13]:
pe2l

,SCORE,REASON1,REASON2,TRUE_CLASS
176,1.00,hue <= 0.82,flavanoids <= 1.01,2
155,1.00,hue <= 0.82,flavanoids <= 1.01,2
174,1.00,hue <= 0.82,flavanoids <= 1.01,2
148,0.98,hue <= 0.82,flavanoids <= 1.01,2
169,0.94,hue <= 0.82,flavanoids <= 1.01,2
163,0.94,hue <= 0.82,flavanoids <= 1.01,2
171,0.91,hue <= 0.82,flavanoids <= 1.01,2
131,0.90,hue <= 0.82,od280/od315_of_diluted_wines <= 1.85,2
170,0.85,hue <= 0.82,flavanoids <= 1.01,2
136,0.81,hue <= 0.82,flavanoids <= 1.01,2


## Explanations by tree interpretation

In [14]:
import tree_explainer

### Explanations for top cases predicted to belong to class 0

In [15]:
%%time
pe0t = tree_explainer.predict_explain(clf[0], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 330 ms


In [16]:
pe0t

,SCORE,REASON1,REASON2,TRUE_CLASS
58,1.00,proline > 1010.00,2.57 < total_phenols <= 3.41,0
57,1.00,proline > 1010.00,2.57 < total_phenols <= 3.28,0
0,1.00,proline > 1010.00,2.57 < total_phenols <= 3.28,0
26,0.98,proline > 1010.00,2.57 < total_phenols <= 3.28,0
49,0.98,proline > 1045.00,2.66 < total_phenols <= 3.28,0
31,0.98,proline > 1045.00,2.66 < total_phenols <= 3.28,0
51,0.96,proline > 1082.50,flavanoids > 2.70,0
47,0.96,proline > 895.00,2.57 < total_phenols <= 3.28,0
9,0.95,proline > 1010.00,2.57 < total_phenols <= 3.28,0
13,0.95,proline > 1010.00,flavanoids > 2.70,0


### Explanations for top cases predicted to belong to class 1

In [17]:
%%time
pe1t = tree_explainer.predict_explain(clf[1], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 288 ms


In [18]:
pe1t

,SCORE,REASON1,REASON2,TRUE_CLASS
128,1.00,color_intensity <= 3.26,proline <= 375.00,1
97,1.00,color_intensity <= 3.43,proline <= 505.00,1
86,1.00,color_intensity <= 3.22,12.16 < alcohol <= 12.21,1
117,0.99,color_intensity <= 3.43,proline <= 375.00,1
113,0.99,color_intensity <= 3.43,proline <= 476.00,1
106,0.99,color_intensity <= 3.43,alcohol <= 12.32,1
114,0.99,color_intensity <= 3.43,proline <= 476.00,1
78,0.98,color_intensity <= 3.43,alcohol <= 12.37,1
99,0.98,color_intensity <= 3.28,proline <= 476.00,1
94,0.98,color_intensity <= 3.28,alcohol <= 12.16,1


### Explanations for top cases predicted to belong to class 2

In [19]:
%%time
pe2t = tree_explainer.predict_explain(clf[2], Xtest).assign(
    TRUE_CLASS = ytest
).sort_values('SCORE', ascending = False).head(20)

Wall time: 274 ms


In [20]:
pe2t

,SCORE,REASON1,REASON2,TRUE_CLASS
176,1.00,hue <= 0.80,flavanoids <= 0.85,2
155,1.00,hue <= 0.80,od280/od315_of_diluted_wines <= 1.78,2
174,1.00,hue <= 0.80,od280/od315_of_diluted_wines <= 1.78,2
148,0.98,hue <= 0.80,od280/od315_of_diluted_wines <= 1.78,2
169,0.94,hue <= 0.76,od280/od315_of_diluted_wines <= 2.01,2
163,0.94,hue <= 0.70,flavanoids <= 0.89,2
171,0.91,hue <= 0.80,1.63 < od280/od315_of_diluted_wines <= 1.78,2
131,0.90,hue <= 0.76,od280/od315_of_diluted_wines <= 1.48,2
170,0.85,hue <= 0.70,flavanoids <= 0.89,2
136,0.81,hue <= 0.84,flavanoids <= 0.89,2


## Comparing the systems

### For class 0

In [21]:
pe0l[['REASON1','REASON2']].join(
    pe0t[['REASON1','REASON2']], 
    lsuffix = '_LIME'
)

,REASON1_LIME,REASON2_LIME,REASON1,REASON2
58,proline > 973.75,flavanoids > 2.81,proline > 1010.00,2.57 < total_phenols <= 3.41
57,proline > 973.75,flavanoids > 2.81,proline > 1010.00,2.57 < total_phenols <= 3.28
0,proline > 973.75,flavanoids > 2.81,proline > 1010.00,2.57 < total_phenols <= 3.28
26,proline > 973.75,flavanoids > 2.81,proline > 1010.00,2.57 < total_phenols <= 3.28
49,proline > 973.75,flavanoids > 2.81,proline > 1045.00,2.66 < total_phenols <= 3.28
31,proline > 973.75,flavanoids > 2.81,proline > 1045.00,2.66 < total_phenols <= 3.28
51,proline > 973.75,flavanoids > 2.81,proline > 1082.50,flavanoids > 2.70
47,proline > 973.75,flavanoids > 2.81,proline > 895.00,2.57 < total_phenols <= 3.28
9,proline > 973.75,flavanoids > 2.81,proline > 1010.00,2.57 < total_phenols <= 3.28
13,proline > 973.75,flavanoids > 2.81,proline > 1010.00,flavanoids > 2.70


### For class 1

In [22]:
pe1l[['REASON1','REASON2']].join(
    pe1t[['REASON1','REASON2']], 
    lsuffix = '_LIME'
)

,REASON1_LIME,REASON2_LIME,REASON1,REASON2
128,color_intensity <= 3.14,proline <= 495.00,color_intensity <= 3.26,proline <= 375.00
97,color_intensity <= 3.14,proline <= 495.00,color_intensity <= 3.43,proline <= 505.00
86,color_intensity <= 3.14,alcohol <= 12.41,color_intensity <= 3.22,12.16 < alcohol <= 12.21
117,color_intensity <= 3.14,proline <= 495.00,color_intensity <= 3.43,proline <= 375.00
113,color_intensity <= 3.14,alcohol <= 12.41,color_intensity <= 3.43,proline <= 476.00
106,alcohol <= 12.41,ash <= 2.20,color_intensity <= 3.43,alcohol <= 12.32
114,color_intensity <= 3.14,proline <= 495.00,color_intensity <= 3.43,proline <= 476.00
78,alcohol <= 12.41,malic_acid <= 1.64,color_intensity <= 3.43,alcohol <= 12.37
99,color_intensity <= 3.14,proline <= 495.00,color_intensity <= 3.28,proline <= 476.00
94,alcohol <= 12.41,proline <= 495.00,color_intensity <= 3.28,alcohol <= 12.16


### For class 2

In [23]:
pe2l[['REASON1','REASON2']].join(
    pe2t[['REASON1','REASON2']], 
    lsuffix = '_LIME'
)

,REASON1_LIME,REASON2_LIME,REASON1,REASON2
176,hue <= 0.82,flavanoids <= 1.01,hue <= 0.80,flavanoids <= 0.85
155,hue <= 0.82,flavanoids <= 1.01,hue <= 0.80,od280/od315_of_diluted_wines <= 1.78
174,hue <= 0.82,flavanoids <= 1.01,hue <= 0.80,od280/od315_of_diluted_wines <= 1.78
148,hue <= 0.82,flavanoids <= 1.01,hue <= 0.80,od280/od315_of_diluted_wines <= 1.78
169,hue <= 0.82,flavanoids <= 1.01,hue <= 0.76,od280/od315_of_diluted_wines <= 2.01
163,hue <= 0.82,flavanoids <= 1.01,hue <= 0.70,flavanoids <= 0.89
171,hue <= 0.82,flavanoids <= 1.01,hue <= 0.80,1.63 < od280/od315_of_diluted_wines <= 1.78
131,hue <= 0.82,od280/od315_of_diluted_wines <= 1.85,hue <= 0.76,od280/od315_of_diluted_wines <= 1.48
170,hue <= 0.82,flavanoids <= 1.01,hue <= 0.70,flavanoids <= 0.89
136,hue <= 0.82,flavanoids <= 1.01,hue <= 0.84,flavanoids <= 0.89
